In [1]:
from fastNLP.io import WeiboSenti100kPipe
from fastNLP.embeddings import BertEmbedding
from fastNLP.models import BertForSequenceClassification
from fastNLP import Trainer, CrossEntropyLoss, AccuracyMetric, Adam
import torch

data_bundle =WeiboSenti100kPipe().process_from_file()
data_bundle.rename_field('chars', 'words')



In total 3 datasets:
	dev has 10000 instances.
	test has 10000 instances.
	train has 99988 instances.
In total 2 vocabs:
	target has 2 entries.
	words has 5933 entries.

In [2]:
# 载入BertEmbedding
embed = BertEmbedding(data_bundle.get_vocab('words'), model_dir_or_name='cn-wwm', include_cls_sep=True)

# 载入模型
model = BertForSequenceClassification(embed, len(data_bundle.get_vocab('target')))



loading vocabulary file C:\Users\Administrator\.fastNLP\embedding\bert-chinese-wwm\vocab.txt
Load pre-trained BERT parameters from file C:\Users\Administrator\.fastNLP\embedding\bert-chinese-wwm\chinese_wwm_pytorch.bin.
Start to generate word pieces for word.
540 words are unsegmented. Among them, 346 added to the BPE vocab.


In [ ]:
# 训练模型
device = 0 if torch.cuda.is_available() else 'cpu'
trainer = Trainer(data_bundle.get_dataset('train'), model,
                  optimizer=Adam(model_params=model.parameters(), lr=2e-5),
                  loss=CrossEntropyLoss(), device=device,
                  batch_size=8, dev_data=data_bundle.get_dataset('dev'),
                  metrics=AccuracyMetric(), n_epochs=2, print_every=1)
trainer.train()



input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 74]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2020-06-18-15-26-27-056010


In [ ]:
# 测试结果
from fastNLP import Tester

tester = Tester(data_bundle.get_dataset('test'), model, batch_size=128, metrics=AccuracyMetric())
tester.test()